# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [72]:
output = open("./some_test_data.txt", 'w')

In [73]:
output.write('123')

3

In [74]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
import datetime
now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'[{now}]:')

[2021-12-11 12:29:49]:


In [6]:
original_write = sys.stdout.write

def my_write(string_text):
    now = datetime.datetime.now()
    now = now.strftime('%Y-%m-%d %H:%M:%S')
    #похоже, что принт делает sys.stdout.write(somestring) + sys.stdout.write('\n')
    #поэтому придется тут вот такую хитрость сделать с символом переноса каретки:
    if string_text == '\n': 
        k = original_write('\n')
        return k
    k = original_write(f'[{now}]: ' + string_text)
    return k
    

sys.stdout.write = my_write

In [7]:
sys.stdout.write('Hello, my friend!')
sys.stdout.write('\n')
sys.stdout.write('Hello, my friend!123')

[2021-12-11 12:29:53]: Hello, my friend!
[2021-12-11 12:29:53]: Hello, my friend!123

43

In [8]:
print('1, 2, 3')

[2021-12-11 12:29:56]: 1, 2, 3


In [9]:
sys.stdout.write = original_write

In [10]:
print('1, 2, 3')

1, 2, 3


Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [11]:
from typing import Callable

In [119]:
def timed_output(func: Callable) -> Callable:
    def wrapper(string_text):
        original_write = sys.stdout.write
        now = datetime.datetime.now()
        now = now.strftime('%Y-%m-%d %H:%M:%S')
        if string_text == '\n': 
            k = original_write('\n')
            func(string_text)
        k = original_write(f'[{now}]: ')
        func(string_text)
    return wrapper

In [120]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [14]:
print_greeting("Nikita")

[2021-12-11 12:30:06]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [132]:
def redirect_output(func: Callable, filepath: str = './function_output.txt') -> Callable:
    def wrapper(func: Callable) -> Callable:
        def inner_wrapper():
            original_write = sys.stdout.write
            output = open(filepath, 'w')
            sys.stdout.write = output.write
            func()
            sys.stdout.write = original_write
        return inner_wrapper        
    return wrapper

In [133]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [134]:
calculate()

In [131]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
